# Collaborative filtering for comprasion

In [1]:
import import_ipynb
import import_data
import Arranging_data_for_comparison
import Similarity_method1
import Similarity_method2

import pandas as pd



importing Jupyter notebook from import_data.ipynb
importing Jupyter notebook from Arranging_data_for_comparison.ipynb
importing Jupyter notebook from Similarity_method1.ipynb
importing Jupyter notebook from Finding_similarity.ipynb
importing Jupyter notebook from Similarity_method2.ipynb


In [2]:
N_RECOMMENDATIONS = 10 # Length of recommendation list
N_MOST_SIMILAR_SESSIONS = N_RECOMMENDATIONS #Option to filter the most similar sessions in Method2 
CSV_FILE_NAME = 'data100000.csv'
REQUIRED_SIMILAR_SEARCHES = 4 #Number of similar searches to filter by


## Import data

In [3]:
data= import_data.import_csv(CSV_FILE_NAME)
import_data.print_data_analysis(data)

Number of users: 54319
Number of sessions: 100000


## Arranging the data

In [4]:
# Creating a DataFrame
data= Arranging_data_for_comparison.create_DataFrame(data)

# Filter sessions that have at least N searches of different cities
filter_sessions_N_dif_searches= Arranging_data_for_comparison.filter_sessions_N_dif_searches(data, REQUIRED_SIMILAR_SEARCHES)
sessions_without_last= Arranging_data_for_comparison.sessions_without_last(filter_sessions_N_dif_searches)
sessions_with_only_last= Arranging_data_for_comparison.sessions_with_only_last(filter_sessions_N_dif_searches)

### Transforming the data

<div>
<img src="img/fig_transforming_data.png" width="600"/>
</div>

In [5]:
utility_matrix=Arranging_data_for_comparison.transforming_data (sessions_without_last)
utility_matrix=Arranging_data_for_comparison.remove_unknown_sessions (utility_matrix,['#NAME?','#VALUE!'])
utility_matrix.head()

city,100,1004,1007,101,1010,1013,1014,1016,1018,102,...,9754,9757,9791,9797,983,985,992,994,995,9999
sessionId,,,,,,,,,,,,,,,,,,,,,
-3iCZ_TqlVb4EMaARPhDH7II,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-4940tPKBLimTqLNUgeVASP8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-4lygmKxUPx6HsdFxWtKvtWG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-4sip_9C5gM6RSgjIEu102Z-,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-5OlHn2qhu5wrJKkNwHhu79l,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Finding similarity

### Method 1: Similarity function A - Counting identical searches

<div>
<img src="img/sim_f_a.png" width="400"/>
</div>

In [6]:
def mehod1(utility_matrix_no_current_session, current_session_searches, n_recommendations):
    utility_matrix_Method_1=Similarity_method1.calc_similarity_method_1(utility_matrix_no_current_session,current_session_searches)
    utility_matrix_Method_1=Similarity_method1.filter_at_least_one_city_in_common(utility_matrix_Method_1)
    utility_matrix_Method_1=Similarity_method1.add_a_total_row(utility_matrix_Method_1)
    # Creating a sorted matrix of cities and the amount of their searches and get recommendations list
    sorted_recommendations_matrix_Method_1=Similarity_method1.get_sorted_city_searches_matrix(utility_matrix_Method_1)
    return Similarity_method1.get_top_X_recommendations_list(sorted_recommendations_matrix_Method_1 , n_recommendations)

### Method 2: Similarity function B

<div>
<img src="img/sim_f_b.png" width="500"/>
</div>

In [7]:
def mehod2(utility_matrix_no_current_session,current_session_searches, n_recommendations):
    utility_matrix_Method_2=Similarity_method2.calc_similarity_method_2(utility_matrix_no_current_session,current_session_searches)
    utility_matrix_Method_2=Similarity_method2.get_N_most_similar_sessions(utility_matrix_Method_2, N_MOST_SIMILAR_SESSIONS)
    #Recommendations list
    sorted_recommendations_matrix_Method_2=Similarity_method2.get_sorted_city_searches_matrix(utility_matrix_Method_2)
    return list(filter(None, sorted_recommendations_matrix_Method_2))[:n_recommendations]


# Call similarity methods for each session

In [8]:
# Call for similarity Method 1 and similarity Method 2
def calc_similarity_for_specific_session(x , stable_utility_matrix, n_recommendations):
    n_x=x.loc[x!=0].to_dict()
    myDict= {
        "searches" : n_x,
        "Recommendations_Method_1" : mehod1(stable_utility_matrix, n_x, n_recommendations),
        "Recommendations_Method_2" : mehod2(stable_utility_matrix, n_x, n_recommendations)           
    }
    return myDict

# Find recommendations for all sessions
def calc_similarity_for_all_sessions(utility_matrix1, f_similarity_methods, stable_utility_matrix, n_recommendations):
    utility_matrix1['comp'] = utility_matrix1.apply(lambda x: f_similarity_methods(x, stable_utility_matrix, n_recommendations), axis=1)
    utility_matrix1=utility_matrix1.filter(['comp'], axis=1)  
    utility_matrix1= pd.DataFrame(utility_matrix1['comp'].values.tolist(), index=utility_matrix1.index) # Split the result into columns of the searches, method 1 recommendations, and method 2 recommendations
    return utility_matrix1

#Add columns of success in each of the methods
def get_index_from_recomendations_lst(x):
    lst_rec1=x['Recommendations_Method_1']
    lst_rec2=x['Recommendations_Method_2']
    l_search=x['Last_search']
    index_lst_rec1=0
    index_lst_rec2=0
    try:
        index_lst_rec1=lst_rec1.index(l_search)+1
    except ValueError: #if last search is not in N recomendations
        index_lst_rec1=0
    try:
        index_lst_rec2=lst_rec2.index(l_search)+1
    except ValueError: #if last search is not in N recomendations
        index_lst_rec2=0
    return index_lst_rec1, index_lst_rec2

#Deep_copy - For convenience - will change later
utility_matrix_with_recomendations= utility_matrix.copy()   
stable_utility_matrix= utility_matrix.copy() 


utility_matrix_with_recomendations=calc_similarity_for_all_sessions(utility_matrix, calc_similarity_for_specific_session, stable_utility_matrix, N_RECOMMENDATIONS)

#Add last search
last_search=sessions_with_only_last.set_index('sessionId').filter(['city'], axis=1).rename(columns = {'city':'Last_search'})
fnl= pd.merge(utility_matrix_with_recomendations, last_search, right_index=True, left_index=True)

#Add columns of success in each of the methods : 0= no matches between the last search an the N recommenadion , 1= the Last_search is the first recomedation
fnl_with_success=fnl.copy() #Deep_copy - For convenience - will change later     
fnl_with_success['Success_M1'],fnl_with_success['Success_M2'] = zip(*fnl_with_success.apply(lambda x: get_index_from_recomendations_lst(x), axis=1))
fnl_with_success

,searches,Recommendations_Method_1,Recommendations_Method_2,Last_search,Success_M1,Success_M2
sessionId,,,,,,
-3iCZ_TqlVb4EMaARPhDH7II,"{'149': 1, '1599': 2, '1647': 1, '1843': 1}","[183, 1467, 1373, 59, 252, 1137, 139, 1085, 12...","[122, 59, 6174, 1137, 1705, 249, 360, 451, 113...",70,0,0
-4940tPKBLimTqLNUgeVASP8,"{'11395': 2, '1238': 2, '16': 3, '1705': 1, '1...","[59, 1373, 1467, 139, 252, 1843, 963, 1004, 25...","[1236, 1373, 1849, 1865, 100, 1016, 1085, 1137...",1647,0,0
-4lygmKxUPx6HsdFxWtKvtWG,"{'1944': 2, '2040': 1, '4473': 1, '59': 2, '61...","[183, 1467, 1373, 139, 252, 963, 251, 1295, 10...","[2025, 2041, 4489, 70, 814, 9999, 100, 1016, 1...",70,0,4
-4sip_9C5gM6RSgjIEu102Z-,"{'100': 1, '1204': 1, '1290': 1, '1730': 1, '2...","[663, 59, 139, 1016, 1373, 1467, 1832, 183, 81...","[1016, 1085, 1137, 1193, 131, 1331, 1373, 139,...",139,3,8
-5OlHn2qhu5wrJKkNwHhu79l,"{'1004': 1, '11157': 1, '11219': 1, '1730': 1,...","[139, 1467, 183, 59, 1373, 963, 252, 1295, 251...","[1325, 183, 195, 3468, 824, 916, 1325, 963, 97...",1943,0,0
...,...,...,...,...,...,...
zz1-s9CF9knFoGIjuYxNE7JB,"{'1290': 1, '2040': 1, '4473': 2, '663': 2, '8...","[814, 1944, 1016, 1832, 59, 139, 916, 2025, 10...","[2025, 2041, 4489, 70, 814, 9999, 100, 1016, 1...",70,0,4
zzHDRQW9_cwTadQhsBtpPjM_,"{'1290': 5, '1832': 1, '1932': 1, '2041': 3, '...","[1373, 4473, 59, 183, 1467, 139, 251, 1843, 10...","[2040, 2043, 814, 856, 916, 2025, 4489, 814, 2...",183,4,0
zzhNtNd8_iguNWJgHa7KnPsg,"{'100': 1, '1004': 1, '1373': 1, '139': 2, '15...","[183, 59, 1467, 1843, 1295, 663, 1085, 1137, 1...","[1295, 131, 1467, 183, 1016, 1085, 1137, 1193,...",183,1,4


# Analysis of results

In [9]:
success_Method1=fnl_with_success['Success_M1'].value_counts().sort_index()
success_Method2=fnl_with_success['Success_M2'].value_counts().sort_index()

match_between_the_methods=fnl_with_success.loc[(fnl_with_success['Success_M1']==fnl_with_success['Success_M2'])].rename(columns = {'Success_M1':'Match between the two methods'})['Match between the two methods'].value_counts().sort_index()
success_Methods=pd.merge(success_Method1,success_Method2, right_index=True, left_index=True)
success_Methods_with_matches=pd.merge(success_Methods,match_between_the_methods, right_index=True, left_index=True)

print("The times the last search did not appear in the N recommendations in either method:")
print(len(fnl_with_success.loc[(fnl_with_success['Success_M1'] == 0) & (fnl_with_success['Success_M2'] == 0)]))

success_Methods_with_matches

The times the last search did not appear in the N recommendations in either method:
2809


,Success_M1,Success_M2,Match between the two methods
0,3211,4650,2809
1,643,195,29
2,436,151,30
3,389,203,11
4,344,201,12
5,290,128,6
6,239,123,4
7,205,125,7
9,160,140,5
10,135,104,3
